In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from statistics import mean
from matplotlib.animation import FuncAnimation, PillowWriter 
from matplotlib import rc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from math import sqrt
from modAL.models import ActiveLearner, CommitteeRegressor
from modAL.disagreement import max_std_sampling
import time
from statistics import mean, median

### Dataset generation

In [ ]:
def generate_config():
    config_option = []
    all_possible_configs = []
    cache_size_list = [' -DSQLITE_DEFAULT_CACHE_SIZE=4000', ' -DSQLITE_DEFAULT_CACHE_SIZE=2000']
    auto_index_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', '']
    page_size_list = [' -DSQLITE_DEFAULT_PAGE_SIZE=512', ' -DSQLITE_DEFAULT_PAGE_SIZE=1024', ' -DSQLITE_DEFAULT_PAGE_SIZE=2048']
    locking_mode_list = [' -DSQLITE_DEFAULT_LOCKING_MODE=0', ' -DSQLITE_DEFAULT_LOCKING_MODE=1']
    omit_feature_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', ' -DSQLITE_OMIT_BTREECOUNT',
                                         ' -DSQLITE_OMIT_BETWEEN_OPTIMIZATION', ' -DSQLITE_OMIT_LIKE_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_LOOKASIDE', ' -DSQLITE_OMIT_QUICKBALANCE', ' -DSQLITE_OMIT_OR_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_SHARED_CACHE', ' -DSQLITE_OMIT_XFER_OPT']
    store_type_list = [' -DSQLITE_TEMP_STORE=0', ' -DSQLITE_TEMP_STORE=1', ' -DSQLITE_TEMP_STORE=2', ' -DSQLITE_TEMP_STORE=3']
    disable_feature_list = [' -DSQLITE_DISABLE_LFS', ' -DSQLITE_DISABLE_DIRSYNC']
    autovacuum_list = [' -DSQLITE_DEFAULT_AUTOVACUUM=0', ' -DSQLITE_DEFAULT_AUTOVACUUM=1']
    eye_2 = np.eye(2)
    eye_3 = np.eye(3)
    eye_4 = np.eye(4)
    eye_9 = np.eye(9)
    for cache_size in cache_size_list:
        for auto_index in auto_index_list:
            for page_size in page_size_list:
                for locking_mode in locking_mode_list:
                    for omit_feature in omit_feature_list:
                        for store_type in store_type_list:
                            for disable_feature in disable_feature_list:
                                for autovacuum in autovacuum_list:
                                    cmd = cache_size
                                    cmd += auto_index
                                    cmd += page_size
                                    cmd += locking_mode
                                    cmd += omit_feature
                                    cmd += store_type
                                    cmd += disable_feature
                                    cmd += autovacuum
                                    v0 = eye_2[cache_size_list.index(cache_size)]
                                    v1 = eye_2[auto_index_list.index(auto_index)]
                                    v2 = eye_3[page_size_list.index(page_size)]
                                    v3 = eye_2[locking_mode_list.index(locking_mode)]
                                    v4 = eye_9[omit_feature_list.index(omit_feature)]
                                    v5 = eye_4[store_type_list.index(store_type)]
                                    v6 = eye_2[disable_feature_list.index(disable_feature)]
                                    v7 = eye_2[autovacuum_list.index(autovacuum)]
                                    all_possible_config = np.concatenate((v0, v1, v2, v3, v4, v5, v6, v7), axis=None)
                                    all_possible_configs.append(all_possible_config)
                                    config_option.append(cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = np.asarray(all_possible_configs)
    return config_features

In [ ]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./sqlite.csv", index_col=0)
results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

# CoMSA

In [ ]:
R2_list = []
time_avg = 0
for seed_num in range(0,21):
    print(seed_num)
    np.random.seed(seed_num)
    sampled_config_ids = list(np.random.randint(3456, size=15))
    initial_idx = np.array_split(sampled_config_ids, 2)

    learner_list = [ActiveLearner(
                        estimator=XGBRegressor(),
                        X_training=config_features[idx], y_training=results[idx]
                )
                for idx in initial_idx]

    # initializing the Committee
    committee = CommitteeRegressor(
        learner_list=learner_list,
        query_strategy=max_std_sampling
    )  
    n_queries = 140
    res_al = []
    start = time.time()
    for idx in range(n_queries):
        X_train = config_features[sampled_config_ids]
        y_train = results[sampled_config_ids]
        X_test = config_features[~np.isin(np.arange(len(config_features)), sampled_config_ids)]
        y_test = results[~np.isin(np.arange(len(config_features)), sampled_config_ids)]
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        relative_error = []
        for i in range(len(X_test)):
            RE=abs(y_test[i]-Y_predict[i])
            relative_error.append(RE*RE)
        res_al.append(sqrt(sum(relative_error)/len(relative_error)))
        query_idx, query_instance = committee.query(config_features)
        sampled_config_ids += list(query_idx)
        committee.teach(config_features[query_idx], results[query_idx])
    end = time.time()
    time_avg = time_avg + end - start 
    print('time:'+ str(end - start))
    R2_list.append(res_al)

In [ ]:
data = []
for i in range(0, len(R2_list)):
    subset = [R2_list[i][idx] for idx in [15,40,65,70,115]]
    data.append(subset)

In [ ]:
data_t = np.transpose(data)
for element in data_t:
    print(mean(element))

In [ ]:
R2 = np.matrix(R2_list, dtype=np.float32)
print(len(R2))

In [ ]:
np.save('../Compare/AL-XG.npy', R2)

## Crossover and mutation

In [ ]:
def get_all_errors(X_train, y_train, regr):
    y_preds = regr.predict(X_train)
    square_errors = (y_train - y_preds)**2
    normalized_errors = (square_errors - square_errors.min()) / (square_errors.max() - square_errors.min())
    return square_errors, normalized_errors

def get_all_distances(X_train):
    dis_metrics = euclidean_distances(X_train, X_train)
    dis_metrics_sum = np.sum(dis_metrics, axis=1)
    normalized_dis = (dis_metrics_sum - dis_metrics_sum.min()) / (dis_metrics_sum.max() - dis_metrics_sum.min())
    return dis_metrics, normalized_dis

def get_ids_by_score(normalized_errors, normalized_dis, dis_metrics, sampled_config_ids, already_crossovered_config_id_list, ratio=0.5):
    # print(ratio)
    all_possible_configs_pairs = []
    all_scores = ratio*normalized_errors + (1-ratio)*normalized_dis
    sorted_idx_desc = all_scores.argsort()[:][::-1]
    # config_id_1 = sorted_idx_desc[0]
    dis_median = np.median(dis_metrics)
    # dis_median = 0
    for config_id_1 in sorted_idx_desc:
        # second_idx_desc = get_second_point_list_by_distance(dis_metrics, config_id_1)
        for config_id in sorted_idx_desc:
            if dis_metrics[config_id_1][config_id] >= dis_median:
                config_id_1_ori, config_id_2_ori = sampled_config_ids[config_id_1], sampled_config_ids[config_id]
                if config_id_1_ori not in already_crossovered_config_id_list and config_id_2_ori not in already_crossovered_config_id_list:
                # if {config_id_1_ori, config_id_2_ori} not in already_crossovered_config_id_list:
                    all_possible_configs_pairs.append([config_id_1_ori, config_id_2_ori])
                    return config_id_1_ori, config_id_2_ori

In [ ]:
def crosssover(config_id_1, config_id_2,  weights, sampled_config_ids, already_crossovered_config_id_list):
    global count_failed
    # dis_metrics, normalized_dis = get_all_weighted_distance(X_train, feature_weights)
    # while True:
    # TODO: need to be improved

    # config_id_1, config_id_2 = get_ids_by_score_new(normalized_errors, normalized_dis, dis_metrics, sampled_config_ids, already_crossovered_config_id_list, ratio)
        # config_id_1, config_id_2 = sampled_config_ids[config_id_1], sampled_config_ids[config_id_2]
        # if config_id_1 < len(config_features) and config_id_2 < len(config_features):
            # break
    count_loop = 0
    new_configs = []
    # refine the weight based on the intuitives
    # weights = feature_weights
    algo_count = 26
    # algo_weight = np.sum(np.absolute(weights)[:algo_count])
    algo_weight = np.max(np.absolute(weights)[:algo_count])
    # cut_index_prob_raw = np.insert(np.absolute(weights)[algo_count:], 0, algo_weight)
    # re_fined_weights = np.concatenate([np.repeat(algo_weight, np.absolute(weights)[algo_count:]])
    cut_index_prob_raw = np.insert(np.absolute(weights)[algo_count:], 0, algo_weight)
    cut_index_prob = cut_index_prob_raw/cut_index_prob_raw.sum()
    already_cut_ids = []

    while True:
        count_loop += 1
        cut_index = np.random.randint(0, 26)
        # cut_index = np.random.choice(5, 1, p=cut_index_prob)[0]
        if cut_index == 4:
            cut_index -= 1
        new_config_1 = np.concatenate([all_possible_configs_cur[config_id_1][:cut_index+1], all_possible_configs_cur[config_id_2][cut_index+1:]])
        new_config_1_ids = np.where((all_possible_configs_cur==new_config_1).all(axis=1))[0]
        if new_config_1_ids.size > 0 and new_config_1_ids[0] not in sampled_config_ids:
            # new_config_1_id = np.where((all_possible_configs_cur==new_config_1).all(axis=1))[0][0]
            # if new_config_1_id not in sampled_config_ids:
            # new_configs.append(new_config_1)
            new_config_2 = np.concatenate([all_possible_configs_cur[config_id_2][:cut_index+1], all_possible_configs_cur[config_id_1][cut_index+1:]])
            new_config_2_ids = np.where((all_possible_configs_cur==new_config_2).all(axis=1))[0]
            if new_config_2_ids.size > 0 and new_config_2_ids[0] not in sampled_config_ids:
                new_configs = [new_config_1, new_config_2]
                break
        if count_loop == 100:
            count_failed +=1
            new_config_1, new_config_2 = np.random.randint(len(config_features), size=2)
            new_configs = [all_possible_configs_cur[config_id_1], all_possible_configs_cur[config_id_2]]
            # already_crossovered_config_id_list.append({config_id_1, config_id_2}) 
            break
    # already_crossovered_config_id_list.append({config_id_1, config_id_2})
    already_crossovered_config_id_list += [config_id_1, config_id_2]
    return new_configs

In [ ]:
def mutation(pre_configs, weights):
    # weights = regr.coef_
    algo_count = 26
    algo_weight = np.max(np.absolute(weights)[:algo_count])
    # algo_weight = np.sum(np.absolute(weights)[:algo_count])
    # algo_weight = np.mean(np.absolute(weights)[:algo_count])
    # np.sum(np.absolute(weights)[:algo_count])
    # re_fined_weights = np.concatenate([np.repeat(algo_weight, np.absolute(weights)[algo_count:]])
    cut_index_prob_raw = np.insert(np.absolute(weights)[algo_count:], 0, algo_weight)
    cut_index_prob = cut_index_prob_raw/cut_index_prob_raw.sum()
    new_configs = []
    config_len = len(pre_configs[0])
    for pre_config in pre_configs:
        # mut_index = np.random.randint(0, config_len)
        mut_index = np.random.choice(len(cut_index_prob), 1, p=cut_index_prob)[0]
        loop = 0
        while loop<500:
            possible_val = np.random.choice(np.unique(all_possible_configs_cur[:, mut_index]))
            new_config = pre_config.copy()
            if new_config[mut_index] != possible_val:
                new_config[mut_index] = possible_val
                new_config_ids = np.where((all_possible_configs_cur[:]==new_config).all(axis=1))[0]
                loop = loop + 1
                if new_config_ids.size > 0:
                    break
        new_configs.append(new_config)
    return new_configs + pre_configs

In [ ]:
r2_mutate=[]
mre_mutate = []
time_avg_mutate = 0
for rand_num in range(0,21,1):
    r_square_list = []
    mre = []
    mean_squared_list = []
    mean_squared_all_list = []
    already_crossovered_config_id_list = []
    coefs_list = []
    count_failed = 0
    
    print("rand_num: "+str(rand_num))
    
    np.random.seed(rand_num)
    sampled_config_ids = np.random.randint(len(config_features), size=8)
    # X_train, X_test, y_train, y_test=train_test_split(config_features,results, test_size=0.8, random_state=rand_num)
    try:
        start = time.time()
        for i in range(32):
    
            X_train = config_features[sampled_config_ids]
            y_train= results[sampled_config_ids]
    
            X_test = config_features[~np.isin(np.arange(len(config_features)), sampled_config_ids)]
            y_test = results[~np.isin(np.arange(len(config_features)), sampled_config_ids)]
    
            regr = XGBRegressor(learning_rate=0.1)
            regr.fit(X_train, y_train)
            y_pred = regr.predict(X_test)
            # coefs_lst.append(regr.coef_)
            y_pred_all = regr.predict(X_test)
            
            relative_error = []
            for i in range(len(X_test)):
                RE = abs(y_test[i]-y_pred[i])
                relative_error.append(RE*RE)
            # mre.append(mean(relative_error)) 
            r_square_list.append(sqrt(sum(relative_error)/len(relative_error)))
            mean_squared_list.append(mean_squared_error(y_test, y_pred))
            mean_squared_all_list.append(mean_squared_error(y_test, y_pred_all))
            feature_weights = regr.feature_importances_
            square_errors, normalized_errors = get_all_errors(X_train, y_train, regr)
            dis_metrics, normalized_dis = get_all_distances(X_train)
            ratio = 0.9
            if np.sum(square_errors) < 0.1:
                ratio = 0
            config_id_1, config_id_2 = get_ids_by_score(normalized_errors, normalized_dis, dis_metrics, sampled_config_ids, already_crossovered_config_id_list, ratio)
    
            new_configs = crosssover(config_id_1, config_id_2, feature_weights, sampled_config_ids, already_crossovered_config_id_list)
            new_configs = mutation(new_configs, feature_weights)
    
            for new_config in new_configs:
                new_config_ids = np.where((all_possible_configs_cur==new_config).all(axis=1))[0]
                if new_config_ids.size > 0:
                    new_config_id = new_config_ids[0]
                    sampled_config_ids = np.append(sampled_config_ids, new_config_id)
        r2_mutate.append(r_square_list)
        # mre_mutate.append(mre)
        end = time.time()
        time_avg_mutate = time_avg_mutate + end - start 
        print(end - start)
    except:
        print("An exception occurred")
    

In [ ]:
data = []
for i in range(0, len(r2_mutate)):
    subset = [r2_mutate[i][idx] for idx in [4,10,17,23,29]]
    data.append(subset)

data_t = np.transpose(data)
for element in data_t:
    print(mean(element))

In [ ]:
R2 = np.matrix(r2_mutate, dtype=np.float32)
print(len(R2))

In [ ]:
np.save('../Compare/Mutate.npy', R2)

## Random - Baseline

In [ ]:
r2_random = []
time_random = 0
for rand_num in range(0,101,5): 
    print(rand_num)
    start = time.time()
    np.random.seed(rand_num)
    sampled_config_ids_rand = list(np.random.randint(len(X_train), size=20))

    model=RandomForestRegressor()

    n_queries = 140
    res_al_rand = []
    for idx in range(n_queries):
        X_train = config_features[sampled_config_ids_rand]
        y_train = results[sampled_config_ids_rand]
        X_test = config_features[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
        y_test = results[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
    
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        relative_error = []
        for i in range(len(X_test)):
            RE = abs(y_test[i]-Y_predict[i])
            relative_error.append(RE*RE)
        # res_al_rand.append(mean(relative_error))
        res_al_rand.append(sqrt(sum(relative_error)/len(relative_error)))
        query_idx = np.random.randint(len(config_features), size=1)
        sampled_config_ids_rand += list(query_idx)
    end = time.time()
    print(end-start)
    time_random += end-start
    r2_random.append(res_al_rand)

In [ ]:
data = []
for i in range(0, len(r2_random)):
    subset = [r2_random[i][idx] for idx in [5,30,55,80,105]]
    data.append(subset)

data_t = np.transpose(data)
for element in data_t:
    print(mean(element))

In [ ]:
R2 = np.matrix(r2_random, dtype=np.float32)
np.save('../Compare/Random.npy', R2)

# FLASH

In [ ]:
index = np.load('../Compare/FLASH/sqlite_AllNumeric.npy')

In [ ]:
r2_list=[]
for l in [21, 46, 71, 96, 121]:
    r2 = []
    for i in range(0, len(index)):
        X_train = config_features[index[i,:l]]
        y_train = results[index[i,:l]]
        X_test = config_features[~np.isin(np.arange(len(config_features)), index[i,:l])]
        y_test = results[~np.isin(np.arange(len(config_features)), index[i,:l])]
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        relative_error = []
        for i in range(len(X_test)):
            relative_error.append(abs(y_test[i]-Y_predict[i])/y_test[i])
        # r2.append(mean(relative_error))
        r2.append(model.score(X_test,y_test))
    r2_list.append(r2)

In [ ]:
R2 = np.matrix(r2_list, dtype=np.float32)
np.save('../Compare/FLASH_sqlite.npy', R2)

# SPL

In [ ]:
index = np.load('../Compare/SPL_Conqueror/sqlite/divDistBased.npy')

In [ ]:
r2_list=[]
for l in [21, 46, 71, 96, 121]:
    print(l)
    r2 = []
    for i in range(0, len(index)):
        np.random.seed(i)
        sampled_config_ids_rand = np.array(np.random.randint(len(X_train), size=20))
        sampled_config_ids_rand = np.concatenate((sampled_config_ids_rand, index[i,:l]))
        X_train = config_features[sampled_config_ids_rand]
        y_train = results[sampled_config_ids_rand]
        X_test = config_features[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
        y_test = results[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        relative_error = []
        for i in range(len(X_test)):
            RE = abs(y_test[i]-Y_predict[i])
            relative_error.append(RE*RE)
        # r2.append(mean(relative_error))
        r2.append(sqrt(sum(relative_error)/len(relative_error)))
    r2_list.append(r2)

In [ ]:
for element in r2_list:
    print(mean(element))

In [ ]:
R2 = np.matrix(r2_list, dtype=np.float32)
np.save('../Compare/SPL_Conqueror/SPL_henard_sqlite.npy', R2)

# NsbS

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
index = np.load('../Compare/NsbS/sqlite/NsbS.npy')
r2_list=[]
for l in [15,40,65,90,115]:
    r2 = []
    for i in range(0, len(index)):
        np.random.seed(i)
        sampled_config_ids_rand = np.array(np.random.randint(len(X_train), size=20))
        sampled_config_ids_rand = np.concatenate((sampled_config_ids_rand, index[i,:l]))
        X_train = config_features[sampled_config_ids_rand]
        y_train = results[sampled_config_ids_rand]
        X_test = config_features[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
        y_test = results[~np.isin(np.arange(len(config_features)), sampled_config_ids_rand)]
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        relative_error = []
        # print(r2_score(y_test, Y_predict))
        for i in range(len(X_test)):
            RE = abs(y_test[i]-Y_predict[i])
            relative_error.append(RE*RE)
        r2.append(sqrt(sum(relative_error)/len(relative_error)))
        # print(r2)
        #r2.append(r2_score(y_test, Y_predict))
    r2_list.append(r2)

In [ ]:
for element in r2_list:
    print(mean(element))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Y_predict, y_test,'o')
plt.show()

In [ ]:
R2 = np.matrix(r2_list, dtype=np.float32)
np.save('../Compare/NsbS/sqlite/result.npy', R2)